# PyCity Schools Analysis

From Pandas Challenge
UC Davis Data Analysis Bootcamp
George Bigham


In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np


# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# calculate values
tot_schools   = len(school_data)
tot_students  = len(student_data)
tot_budget    = school_data["budget"].sum()
avg_math      = student_data["math_score"].mean()
avg_read      = student_data["reading_score"].mean()
math_pass_per = len(student_data[student_data.math_score >= 70])/tot_students*100
read_pass_per = len(student_data[student_data.reading_score >= 70])/tot_students*100
overall_pass  = (math_pass_per + read_pass_per)/2

# create dataframe
district_data = pd.DataFrame(
                    {"Total Schools" : [tot_schools],
                     "Total Students" : [tot_students],
                     "Total Budget" : [tot_budget],
                     "Average Math Score" : [avg_math],
                     "Average Reading Score" : [avg_read],
                     "% Passing Math" : [math_pass_per],
                     "% Passing Reading" : [read_pass_per],
                     "Overall Passing Rate" : [overall_pass]})

# display dataframe
district_data

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

In [4]:
# the following creates the school_summary dataframe as described above

# Create Per Student Budget column and group
school_data['Per Student Budget'] = school_data['budget'] / school_data['size']
g = school_data_complete.groupby('School ID')

# Calculate averages 
avg_g = g["math_score", "reading_score"].mean()
m_passing = g.apply(lambda x: x[x['math_score'] >= 70]['math_score'].count() / x['math_score'].count()*100 )
r_passing = g.apply(lambda x: x[x['reading_score'] >= 70]['reading_score'].count()/x['reading_score'].count()*100)

# Format new data
avg_g = avg_g.rename(columns= {"math_score" : "Average Math Score", "reading_score" : "Average Reading Score"})
m_passing = m_passing.to_frame()
m_passing = m_passing.rename(columns= {"School ID": "School Id", 0:"% Passing Math"})
r_passing = r_passing.to_frame()
r_passing = r_passing.rename(columns= {"School ID": "School Id", 0:"% Passing Reading"})

# Merge data, one column at a time
m1 = pd.merge(school_data, avg_g, on = "School ID")
m2 = pd.merge(m1, m_passing, on = "School ID")
school_summary = pd.merge(m2, r_passing, on = "School ID")

# Calculate Overall collumn and drop ID
school_summary['Overall Passing Rate'] = (school_summary['% Passing Math'] + school_summary['% Passing Reading'])/2
school_summary = school_summary.drop(columns = ['School ID'])

* Sort and display the top five schools in overall passing rate

In [5]:
top_five = school_summary.sort_values(by = 'Overall Passing Rate', ascending=False)
top_five.head()

,school_name,type,size,budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
14,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
bot_five = school_summary.sort_values(by = 'Overall Passing Rate')
bot_five.head()

,school_name,type,size,budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [10]:
g = school_data_complete.groupby('school_name')

ninth = g.apply(lambda x: x[x['grade'] == "9th"]['math_score'].mean() )
tenth = g.apply(lambda x: x[x['grade'] == "10th"]['math_score'].mean() )
eleventh = g.apply(lambda x: x[x['grade'] == "11th"]['math_score'].mean() )
twelth = g.apply(lambda x: x[x['grade'] == "12th"]['math_score'].mean() )

ninth = ninth.to_frame()
tenth = tenth.to_frame()
eleventh = eleventh.to_frame()
twelth = twelth.to_frame()

ninth = ninth.rename(columns = {0:"9th"})
tenth = tenth.rename(columns = {0:"10th"})
eleventh = eleventh.rename(columns = {0:"11th"})
twelth = twelth.rename(columns = {0:"12th"})

m1 = pd.merge(ninth, tenth, on="school_name")
m2 = pd.merge(m1, eleventh, on="school_name")
math_by_grade = pd.merge(m2, twelth, on="school_name")
math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [11]:
# 
g = school_data_complete.groupby('school_name')

# Calculate new column
ninth = g.apply(lambda x: x[x['grade'] == "9th"]['reading_score'].mean() )
tenth = g.apply(lambda x: x[x['grade'] == "10th"]['reading_score'].mean() )
eleventh = g.apply(lambda x: x[x['grade'] == "11th"]['reading_score'].mean() )
twelth = g.apply(lambda x: x[x['grade'] == "12th"]['reading_score'].mean() )

# Change series to dataframes, seems like 
ninth = ninth.to_frame()
tenth = tenth.to_frame()
eleventh = eleventh.to_frame()
twelth = twelth.to_frame()

# Format column names
ninth = ninth.rename(columns = {0:"9th"})
tenth = tenth.rename(columns = {0:"10th"})
eleventh = eleventh.rename(columns = {0:"11th"})
twelth = twelth.rename(columns = {0:"12th"})

# Combine columns
m1 = pd.merge(ninth, tenth, on="school_name")
m2 = pd.merge(m1, eleventh, on="school_name")
reading_by_grade = pd.merge(m2, twelth, on="school_name")
reading_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [12]:
# Binning based on per student spending quartiles
spending_bins = [0, 592, 628, 641, 656]
group_names = ["<$592", "$592-628", "$628-641", "$641-656"]

In [13]:
school_summary['Spending Ranges(Per Student)'] = pd.cut(school_summary["Per Student Budget"], spending_bins, labels=group_names)
g_spending = school_summary.groupby("Spending Ranges(Per Student)")
g_spending[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges(Per Student),,,,,
<$592,83.455399,83.933814,93.460096,96.610877,95.035486
$592-628,81.899826,83.155286,87.133538,92.718205,89.925871
$628-641,78.990942,81.917212,75.209078,86.089386,80.649232
$641-656,77.023555,80.957446,66.701010,80.675217,73.688113


## Scores by School Size

* Perform the same operations as above, based on school size.

In [14]:
# default bins with one extra bin for very large
size_bins = [0, 1000, 2000, 3000, 5000]
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-3000)", "Very Large (>3000)"]

In [15]:
school_summary['School Size'] = pd.cut(school_summary["size"], size_bins, labels=size_names)
g_size = school_summary.groupby("School Size")
g_size[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-3000),78.429493,81.769122,73.462428,84.473577,78.968003
Very Large (>3000),77.063340,80.919864,66.464293,81.059691,73.761992


## Scores by School Type

* Perform the same operations as above, based on school type.

In [16]:
g_type = school_summary.groupby("type")
g_type[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


Conclusions:

The percent passing math and reading for charter schools is much higher for charter schools, but the difference between raw scores is much less.  Specifically math raw score difference is less then 6 points between Charter and District but passing rate is alsmost 30% higher.  Diffence between raw reading score averages is only 3 points, but there is a 16% difference in passing rate.  This shows that the choice of pass/fail cut point (70 pts) has a strong influence on the ovall pass rate. 

There seems to be very little difference between the small and medium categories of school size, and also little difference between large and very large.  This indicates that having more or less than 2000 students is an indicator of better or worse test scores.

The spending per student and performance seem to be inversely related.  These seems paradoxical, since we assume spending money should raise scores.  However, from my domain knowledge as a former high school teacher, often the most expensive students are special needs students who require special educators and sometimes expensive accomadations to provide the best education for this population.  In general we should isolate what is causing the difference in expenses.  It could be school type and size are corrolated with spending, making the low score a spurious connection.  Further research would be required, but it certainly seems implausable that cuting spending directly raises test scores. 

